# 2. Data Wrangling

### Table of contents  

- Pandas Dataframes
   - Creation
   - Properties
   - Accessing
   - Extension
   - Missing values
   - Deletion
   - Operations
      - Arithmetic
      - Logical
      - String Functions
      - Arbitrary Functions
      - Statistical
      - Ordering
   - Data Transformations
      - Group By
      - Pivot
      - Melt
      - Join
   - Time Series
   - Hierarchical Indices
   - Loading and Saving Dataframes


- Further feading

## 2b. Pandas Dataframes

Dataframes are easy-to-use, functionality-packed data structures for data handling and analysis. They can be thought of as non-homogenous matrices  for row/column labeled data, which also offer a lot of extra functionality.

In [1]:
import numpy as np
import pandas as pd  # the usual abbreviation

### Creation

Instantiate from a 2D array:

In [2]:
m = [
    [5, 2, 3],
    [4, 5, 1],
    [7, 1, 2],
    [6, 2, 9],
]

In [3]:
pd.DataFrame(m)

,0,1,2
0,5,2,3
1,4,5,1
2,7,1,2
3,6,2,9


When working with labeled data, provide a dict of `column_name : value_for_each_row`:

In [4]:
students = pd.DataFrame({
    'height':    np.random.randint(150, 200, size=5),
    'weight':    np.random.randint(50,  100, size=5),
    'graduated': np.random.random(size=5) > .5,  # same as np.random.randint(0, 1, size=10).astype(bool)
})

Each row is an observation (i.e.: a student), each column is a variable (i.e.: a measurement):

In [5]:
students

,height,weight,graduated
0,184,77,False
1,175,57,False
2,179,95,True
3,170,92,False
4,150,79,False


_Note_: since randomness is involved, when you run this notebook, you'll likely see different results.

Dataframes are implicitly indexed by integers, but rows can be assigned more descriptive indices:

In [6]:
students.index = list('abcde')

In [7]:
students

,height,weight,graduated
a,184,77,False
b,175,57,False
c,179,95,True
d,170,92,False
e,150,79,False


**💪 Exercise**: create a new dataframe, `food_stats`, which contains ratings, on a 1-3 scale for `tasty`, `healthy` and whether you had it recently (`had_recently`) for the following types of food: `pizza`, `carrot`, `chocolate`, `banana`:

In [8]:
food_stats = pd.DataFrame({
    'tasty':        [2, 1, 3, 2],
    'healthy':      [1, 3, 2, 3],
    'had_recently': [True, True, False, True],
}, index=['pizza', 'carrot', 'chocolate', 'banana'])

food_stats

,tasty,healthy,had_recently
pizza,2,1,True
carrot,1,3,True
chocolate,3,2,False
banana,2,3,True


### Properties

Shape and data type are extended to dataframes as well:

In [9]:
len(students)  # size of first dimension, i.e.: number of rows

5

In [10]:
students.shape

(5, 3)

In [11]:
students.dtypes

height       int64
weight       int64
graduated     bool
dtype: object

In [12]:
students.height.astype(float)

a    184.0
b    175.0
c    179.0
d    170.0
e    150.0
Name: height, dtype: float64

**💪 Exercise**: check the `shape` and `dtypes` of your `food_stats`:

In [13]:
food_stats.shape

(4, 3)

In [14]:
food_stats.dtypes

tasty           int64
healthy         int64
had_recently     bool
dtype: object

### Accessing

Access rows elements by index:

In [15]:
students

,height,weight,graduated
a,184,77,False
b,175,57,False
c,179,95,True
d,170,92,False
e,150,79,False


In [16]:
students.loc['a']  # the observations for student A

height         184
weight          77
graduated    False
Name: a, dtype: object

Access rows by their position, regardless of index name:

In [17]:
students.iloc[0]

height         184
weight          77
graduated    False
Name: a, dtype: object

Slices are extended row-wise:

In [18]:
students[:3]

,height,weight,graduated
a,184,77,False
b,175,57,False
c,179,95,True


In [19]:
students.head(3)

,height,weight,graduated
a,184,77,False
b,175,57,False
c,179,95,True


---

Access a specific column:

In [20]:
students['graduated']

a    False
b    False
c     True
d    False
e    False
Name: graduated, dtype: bool

**Tip**: sometimes, reserved keys cannot be used to access certain columns (e.g.: `size`), so you must use the dict-like syntax

In [21]:
students.graduated  # we can also access column-wise

a    False
b    False
c     True
d    False
e    False
Name: graduated, dtype: bool

Access multiple columns at once:

In [22]:
columns = ['weight', 'height']
students[columns]

,weight,height
a,77,184
b,57,175
c,95,179
d,92,170
e,79,150


Masking is performed row-wise by providing a boolean array with the same length as the number of rows:

In [23]:
students[students.graduated]

,height,weight,graduated
c,179,95,True


---

Randomly sample rows:

In [24]:
students.sample(3)  # 3 rows

,height,weight,graduated
b,175,57,False
c,179,95,True
e,150,79,False


In [25]:
students.sample(frac=.5)  # half of all rows

,height,weight,graduated
e,150,79,False
c,179,95,True


In [26]:
len(students)

5

---

Iterating over the dataframe defaults to going over its columns:

In [27]:
for column in students:
    print(column)

height
weight
graduated


`iterrows()` is used to iterate over each row:

In [28]:
for student, row in students.iterrows():
    print(student, row.height, row.weight)

a 184 77
b 175 57
c 179 95
d 170 92
e 150 79


**💪 Exercise**: access the `tasty` column, rows 2 through 4 of `food_stats`:

In [29]:
food_stats[2:4]['tasty']

chocolate    3
banana       2
Name: tasty, dtype: int64

**💪 Exercise**: select rows for students yet to graduate:

In [30]:
students[~students.graduated]

,height,weight,graduated
a,184,77,False
b,175,57,False
d,170,92,False
e,150,79,False


### Extension

To add a new row, just specify it's index (`loc`) or positional index (`iloc`) and the value for each column:

In [31]:
students.loc['x'] = (170, 70, True)

In [32]:
students

,height,weight,graduated
a,184,77,False
b,175,57,False
c,179,95,True
d,170,92,False
e,150,79,False
x,170,70,True


**ℹ️ Tip**: even though `df.iloc[len(df) - 1] = ...` can be used to append at the end of any dataframe, this is not recommended. If you wish to create a dataframe iteratively, instead of appending each element, store them in a different container and convert the data to a dataframe at the end. This is also the reason why there is no `append` function for dataframes.

To add a new column, assign it directly to the dataframe and specify a value for each row:

In [33]:
students['age'] = np.random.randint(18, 24, size=len(students))

In [34]:
students

,height,weight,graduated,age
a,184,77,False,18
b,175,57,False,22
c,179,95,True,22
d,170,92,False,22
e,150,79,False,23
x,170,70,True,22


**ℹ️ Tip**: this syntax must be used for column creation. `df.column` only works for accessing existing columns.

Create a column based on another:

In [35]:
students['can_ride'] = (students.height > 170)  # "you must be this tall to ride the roller coaster"

In [36]:
students

,height,weight,graduated,age,can_ride
a,184,77,False,18,True
b,175,57,False,22,True
c,179,95,True,22,True
d,170,92,False,22,False
e,150,79,False,23,False
x,170,70,True,22,False


---

Add multiple new rows, from another dataframe:

In [37]:
new_students = pd.DataFrame({
    'height': [160, 180],
    'weight': [ 60,  80],
})

new_students

,height,weight
0,160,60
1,180,80


In [38]:
students = pd.concat([students, new_students], sort=False)
students

,height,weight,graduated,age,can_ride
a,184,77,False,18.0,True
b,175,57,False,22.0,True
c,179,95,True,22.0,True
d,170,92,False,22.0,False
e,150,79,False,23.0,False
x,170,70,True,22.0,False
0,160,60,NaN,NaN,NaN
1,180,80,NaN,NaN,NaN


_Note_ it's `pd.concat` but `np.concatenate`

Add multiple new columns, from another dataframe:

In [39]:
n_students = len(students)

new_info = pd.DataFrame({
    'fav_number':   np.random.randint(0, 100, size=n_students),
    'fav_icecream': np.random.choice(['vanilla', 'chocolate', 'strawberry'], size=n_students),
})

In [40]:
new_info.index = students.index  # set the same index, to make merging (and viewing) easier
new_info

,fav_number,fav_icecream
a,77,vanilla
b,79,strawberry
c,10,strawberry
d,6,vanilla
e,62,strawberry
x,27,chocolate
0,41,vanilla
1,41,vanilla


In [41]:
students = students.merge(new_info, left_index=True, right_index=True)
students

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
a,184,77,False,18.0,True,77,vanilla
b,175,57,False,22.0,True,79,strawberry
c,179,95,True,22.0,True,10,strawberry
d,170,92,False,22.0,False,6,vanilla
e,150,79,False,23.0,False,62,strawberry
x,170,70,True,22.0,False,27,chocolate
0,160,60,NaN,NaN,NaN,41,vanilla
1,180,80,NaN,NaN,NaN,41,vanilla


**💪 Exercise**: add a new entry for `hamburger` in the `food_stats` dataframe:

In [42]:
food_stats.loc['hamburger'] = (3, 1, False)
food_stats

,tasty,healthy,had_recently
pizza,2,1,True
carrot,1,3,True
chocolate,3,2,False
banana,2,3,True
hamburger,3,1,False


### Missing Values

Since we only provided the `height` and `weight` measurements, for the new students, `nan` (not a number) is placed by default for the other columns:

In [43]:
students

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
a,184,77,False,18.0,True,77,vanilla
b,175,57,False,22.0,True,79,strawberry
c,179,95,True,22.0,True,10,strawberry
d,170,92,False,22.0,False,6,vanilla
e,150,79,False,23.0,False,62,strawberry
x,170,70,True,22.0,False,27,chocolate
0,160,60,NaN,NaN,NaN,41,vanilla
1,180,80,NaN,NaN,NaN,41,vanilla


Missing value detection:

In [44]:
pd.isna(students)

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
a,False,False,False,False,False,False,False
b,False,False,False,False,False,False,False
c,False,False,False,False,False,False,False
d,False,False,False,False,False,False,False
e,False,False,False,False,False,False,False
x,False,False,False,False,False,False,False
0,False,False,True,True,True,False,False
1,False,False,True,True,True,False,False


Column-wise:

In [45]:
pd.isna(students.age)

a    False
b    False
c    False
d    False
e    False
x    False
0     True
1     True
Name: age, dtype: bool

The easiest method for handling missing data is dropping the observation alltogether:

In [46]:
students.dropna()

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
a,184,77,False,18.0,True,77,vanilla
b,175,57,False,22.0,True,79,strawberry
c,179,95,True,22.0,True,10,strawberry
d,170,92,False,22.0,False,6,vanilla
e,150,79,False,23.0,False,62,strawberry
x,170,70,True,22.0,False,27,chocolate


Another method is filling it with some default value:

In [47]:
students.fillna({
    'graduated': False,
    'can_ride':  False,
    'age':       20,
})

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
a,184,77,False,18.0,True,77,vanilla
b,175,57,False,22.0,True,79,strawberry
c,179,95,True,22.0,True,10,strawberry
d,170,92,False,22.0,False,6,vanilla
e,150,79,False,23.0,False,62,strawberry
x,170,70,True,22.0,False,27,chocolate
0,160,60,False,20.0,False,41,vanilla
1,180,80,False,20.0,False,41,vanilla


More methods for handling missing value will be explored in workshop 4.

**ℹ️ Tip**: the reason why we must use `pd.isna` to identify NANs, instead of `== np.nan` is because `np.nan` is a special element, which, by design, is not equal to anything, not even `np.nan`. Read more about [ternary logic](https://en.wikipedia.org/wiki/Three-valued_logic) and non-[finite numpy numbers](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.isfinite.html#numpy.isfinite).

### Deleting

Delete some columns:

In [48]:
students.drop(['weight', 'age'], axis=1)

,height,graduated,can_ride,fav_number,fav_icecream
a,184,False,True,77,vanilla
b,175,False,True,79,strawberry
c,179,True,True,10,strawberry
d,170,False,False,6,vanilla
e,150,False,False,62,strawberry
x,170,True,False,27,chocolate
0,160,NaN,NaN,41,vanilla
1,180,NaN,NaN,41,vanilla


Delete some rows, index-wise:

In [49]:
students.drop(['a', 'x'], axis=0)

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
b,175,57,False,22.0,True,79,strawberry
c,179,95,True,22.0,True,10,strawberry
d,170,92,False,22.0,False,6,vanilla
e,150,79,False,23.0,False,62,strawberry
0,160,60,NaN,NaN,NaN,41,vanilla
1,180,80,NaN,NaN,NaN,41,vanilla


---

Deleting rows based on a boolean filtering is done by masking:

In [50]:
mask = (students.age > 21)  # filter underage ones
students[mask]

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
b,175,57,False,22.0,True,79,strawberry
c,179,95,True,22.0,True,10,strawberry
d,170,92,False,22.0,False,6,vanilla
e,150,79,False,23.0,False,62,strawberry
x,170,70,True,22.0,False,27,chocolate


Dropping duplicate rows can be done using `drop_duplicate`, but since our dataframe contains no such rows, we will restrict it to duplicates on just the `can_ride` and `fav_icecream` columns:

In [51]:
students

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
a,184,77,False,18.0,True,77,vanilla
b,175,57,False,22.0,True,79,strawberry
c,179,95,True,22.0,True,10,strawberry
d,170,92,False,22.0,False,6,vanilla
e,150,79,False,23.0,False,62,strawberry
x,170,70,True,22.0,False,27,chocolate
0,160,60,NaN,NaN,NaN,41,vanilla
1,180,80,NaN,NaN,NaN,41,vanilla


In [52]:
students.drop_duplicates(subset=['can_ride', 'fav_icecream'], keep='first')

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
a,184,77,False,18.0,True,77,vanilla
b,175,57,False,22.0,True,79,strawberry
d,170,92,False,22.0,False,6,vanilla
e,150,79,False,23.0,False,62,strawberry
x,170,70,True,22.0,False,27,chocolate
0,160,60,NaN,NaN,NaN,41,vanilla


---

Deleting, much like any operation in the next subsection, does not operate _in place_. This means the `drop` function returns a new dataframe object, which is created by (deep) copying the original one and applying the operation on it. Writing your own functions in such a way helps with preventing unexpected and hard to trace side effects. It also allows for method chaining `df.transpose().mean().round()`. Read more about [functional programming](https://hackernoon.com/learn-functional-python-in-10-minutes-to-2d1651dece6f) and [immutability in Python](https://www.pythonforthelab.com/blog/mutable-and-immutable-objects/).

If you do wish to "update" the same object, assign the result to the same variable:

In [53]:
students = students.dropna()

Some functions also offer the `inplace` option:

In [54]:
students.dropna(inplace=True)  # equivalent to above

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


**💪 Exercise**: Drop the `graduated` column from `students` (not in place!):

In [55]:
students.drop('graduated', axis=1)

,height,weight,age,can_ride,fav_number,fav_icecream
a,184,77,18.0,True,77,vanilla
b,175,57,22.0,True,79,strawberry
c,179,95,22.0,True,10,strawberry
d,170,92,22.0,False,6,vanilla
e,150,79,23.0,False,62,strawberry
x,170,70,22.0,False,27,chocolate


### Dataframe Operations

Array-wise functions and operations are naturally extended to dataframes

**ℹ️ Tip**: a dataframe is composed by multiple `pd.Series`. Each column can be a series, and each row can be a series. A series is a "labeled list" — where each `value` has an `index`.

In [56]:
students

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
a,184,77,False,18.0,True,77,vanilla
b,175,57,False,22.0,True,79,strawberry
c,179,95,True,22.0,True,10,strawberry
d,170,92,False,22.0,False,6,vanilla
e,150,79,False,23.0,False,62,strawberry
x,170,70,True,22.0,False,27,chocolate


#### Arithmetic

In [57]:
students.weight - 10  # if only losing weight was this easy 😅

a    67
b    47
c    85
d    82
e    69
x    60
Name: weight, dtype: int64

In [58]:
students.height + students.weight  # note that the resulting series still has the same indices

a    261
b    232
c    274
d    262
e    229
x    240
dtype: int64

In [59]:
students.sum()

height                                                       1028
weight                                                        470
graduated                                                       2
age                                                           129
can_ride                                                        3
fav_number                                                    261
fav_icecream    vanillastrawberrystrawberryvanillastrawberrych...
dtype: object

In [60]:
students.mean()

height        171.333333
weight         78.333333
graduated       0.333333
age            21.500000
can_ride        0.500000
fav_number     43.500000
dtype: float64

**ℹ️ Tip**: The sum of the boolean series `graduated` is the number of people that graduated. The mean of `graduated` is the sum divided by the total number of students, which is precisely the percentage of students that graduated.

Most operations accept an `axis` argument, which can either be `0` (column-wise, default), or `1` (row-wise):

In [61]:
students.mean(axis=1)  # the average for each student, of their height, weight and graduation status.. which doesn't make much sense

a    89.00
b    83.25
c    76.50
d    72.50
e    78.50
x    72.25
dtype: float64

**💪 Exercise**: get the sums of the `tasty` and `healthy` ratings in `food_stats`:

In [62]:
food_stats[['tasty', 'healthy']].sum()

tasty      11
healthy    10
dtype: int64

#### Logical

In [63]:
students.age > 21

a    False
b     True
c     True
d     True
e     True
x     True
Name: age, dtype: bool

In [64]:
students.graduated & ~students.can_ride  # students who graduated but cannot ride

a    False
b    False
c     True
d    False
e    False
x     True
dtype: bool

In [65]:
students.graduated.any()

True

In [66]:
students.graduated.all()

False

In [67]:
students == 21

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
a,False,False,False,False,False,False,False
b,False,False,False,False,False,False,False
c,False,False,False,False,False,False,False
d,False,False,False,False,False,False,False
e,False,False,False,False,False,False,False
x,False,False,False,False,False,False,False


**💪 Exercise**: which tasty (`tasty` score $\ge 2$) food items have you had recently?

In [68]:
food_stats[(food_stats.tasty >= 2) & food_stats.had_recently]

,tasty,healthy,had_recently
pizza,2,1,True
banana,2,3,True


#### String Functions

The raw string values of textual variables can be accessed with `.str`:

In [69]:
students.fav_icecream

a       vanilla
b    strawberry
c    strawberry
d       vanilla
e    strawberry
x     chocolate
Name: fav_icecream, dtype: object

In [70]:
students.fav_icecream.str.title()

a       Vanilla
b    Strawberry
c    Strawberry
d       Vanilla
e    Strawberry
x     Chocolate
Name: fav_icecream, dtype: object

In [71]:
students.fav_icecream.str.replace('straw', 'banned-')

a         vanilla
b    banned-berry
c    banned-berry
d         vanilla
e    banned-berry
x       chocolate
Name: fav_icecream, dtype: object

In [72]:
students.fav_icecream.str.contains('e')

a    False
b     True
c     True
d    False
e     True
x     True
Name: fav_icecream, dtype: bool

**💪 Exercise**: in `food_stats`, make the name of the food items (the `index`) UPPERCASE:

In [73]:
food_stats.index = food_stats.index.str.upper()
food_stats

,tasty,healthy,had_recently
PIZZA,2,1,True
CARROT,1,3,True
CHOCOLATE,3,2,False
BANANA,2,3,True
HAMBURGER,3,1,False


#### Arbitrary Functions

In [74]:
students.height.apply(lambda w: (w // 10) * 10)  # apply to each element of a column

a    180
b    170
c    170
d    170
e    150
x    170
Name: height, dtype: int64

In [75]:
students.apply(lambda row: row.height + row.weight, axis=1)  # apply row-wise

a    261
b    232
c    274
d    262
e    229
x    240
dtype: int64

In [76]:
def relabel_boolean(x):
    # if the argument is not a boolean, leave it as it is
    if type(x) is not bool:
        return x
    return 'yes' if x is True else 'no'

In [77]:
students.applymap(relabel_boolean)  # apply element-wise

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
a,184,77,no,18.0,yes,77,vanilla
b,175,57,no,22.0,yes,79,strawberry
c,179,95,yes,22.0,yes,10,strawberry
d,170,92,no,22.0,no,6,vanilla
e,150,79,no,23.0,no,62,strawberry
x,170,70,yes,22.0,no,27,chocolate


#### Statistical

One of the most useful shortcuts is `describe`, which quickly provides a list of descriptive statistics about each numeric column:

In [78]:
students.describe().round(2)

,height,weight,age,fav_number
count,6.00,6.00,6.00,6.00
mean,171.33,78.33,21.50,43.50
std,11.76,14.08,1.76,33.24
min,150.00,57.00,18.00,6.00
25%,170.00,71.75,22.00,14.25
50%,172.50,78.00,22.00,44.50
75%,178.00,88.75,22.00,73.25
max,184.00,95.00,23.00,79.00


90% of the values are lower than the 90th percentile. This can be used for outlier detection, which will be explored in workshop 4.

In [79]:
students.describe(percentiles=[.1, .5, .9, .95]).round(3)

,height,weight,age,fav_number
count,6.000,6.000,6.000,6.000
mean,171.333,78.333,21.500,43.500
std,11.759,14.081,1.761,33.243
min,150.000,57.000,18.000,6.000
10%,160.000,63.500,20.000,8.000
50%,172.500,78.000,22.000,44.500
90%,181.500,93.500,22.500,78.000
95%,182.750,94.250,22.750,78.500
max,184.000,95.000,23.000,79.000


---

Categorical variables aggregations:

In [80]:
students.fav_icecream.value_counts()

strawberry    3
vanilla       2
chocolate     1
Name: fav_icecream, dtype: int64

In [81]:
students.fav_icecream.unique()

array(['vanilla', 'strawberry', 'chocolate'], dtype=object)

In [82]:
students.fav_icecream.nunique()  # when you need just the amount of unique items, not their actual values

3

---

Two variables have high _pair-wise correlation_ (Pearson) when one is dependent on the other: there is a linear relationship between the two:

In [83]:
students.corr()

,height,weight,age,fav_number
height,1.000000,0.013690,-0.695541,-0.012791
weight,0.013690,1.000000,0.048403,-0.736190
age,-0.695541,0.048403,1.000000,-0.394667
fav_number,-0.012791,-0.736190,-0.394667,1.000000


Two variables have high _covariance_ when they then to show similar behavior: greater values in one correspond to greater values in the other:

In [84]:
students.cov()

,height,weight,age,fav_number
height,138.266667,2.266667,-14.4,-5.0
weight,2.266667,198.266667,1.2,-344.6
age,-14.400000,1.200000,3.1,-23.1
fav_number,-5.000000,-344.600000,-23.1,1105.1


Positive _skewness_ indicates left-leaning distributions:

In [85]:
students.age.skew()

-2.1435978905453736

Positive _kurtosis_ indicates that the distribution has heavy tails and sharp peaks:

In [86]:
students.age.kurt()

5.067637877211238

Read more about [distribution measures](https://support.minitab.com/en-us/minitab-express/1/help-and-how-to/basic-statistics/inference/supporting-topics/data-concepts/how-skewness-and-kurtosis-affect-your-distribution/).

**💪 Exercise**: `describe` your `food_stats`:

In [87]:
food_stats.describe()

,tasty,healthy
count,5.00000,5.0
mean,2.20000,2.0
std,0.83666,1.0
min,1.00000,1.0
25%,2.00000,1.0
50%,2.00000,2.0
75%,3.00000,3.0
max,3.00000,3.0


#### Ordering

Sort by the index:

In [88]:
students

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
a,184,77,False,18.0,True,77,vanilla
b,175,57,False,22.0,True,79,strawberry
c,179,95,True,22.0,True,10,strawberry
d,170,92,False,22.0,False,6,vanilla
e,150,79,False,23.0,False,62,strawberry
x,170,70,True,22.0,False,27,chocolate


In [89]:
students.sort_index()

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
a,184,77,False,18.0,True,77,vanilla
b,175,57,False,22.0,True,79,strawberry
c,179,95,True,22.0,True,10,strawberry
d,170,92,False,22.0,False,6,vanilla
e,150,79,False,23.0,False,62,strawberry
x,170,70,True,22.0,False,27,chocolate


Sort by (a combination of) column(s):

In [90]:
students.sort_values(by=['fav_number', 'age'], ascending=True)

,height,weight,graduated,age,can_ride,fav_number,fav_icecream
d,170,92,False,22.0,False,6,vanilla
c,179,95,True,22.0,True,10,strawberry
x,170,70,True,22.0,False,27,chocolate
e,150,79,False,23.0,False,62,strawberry
a,184,77,False,18.0,True,77,vanilla
b,175,57,False,22.0,True,79,strawberry


Compute each row's rank (average ranks in case of equality):

In [91]:
students.fav_number.rank()

a    5.0
b    6.0
c    2.0
d    1.0
e    4.0
x    3.0
Name: fav_number, dtype: float64

**💪 Exercise**: sort your `food_stats` by `tasty`est first:

In [92]:
food_stats.sort_values(by='tasty', ascending=False)

,tasty,healthy,had_recently
CHOCOLATE,3,2,False
HAMBURGER,3,1,False
PIZZA,2,1,True
BANANA,2,3,True
CARROT,1,3,True


### Data Transformations

Restructuring operations. Same data, but different view, more fitted for the downstream task.

Transpose rows and columns (keeping labels):

In [93]:
students.T

,a,b,c,d,e,x
height,184,175,179,170,150,170
weight,77,57,95,92,79,70
graduated,False,False,True,False,False,True
age,18,22,22,22,23,22
can_ride,True,True,True,False,False,False
fav_number,77,79,10,6,62,27
fav_icecream,vanilla,strawberry,strawberry,vanilla,strawberry,chocolate


Transform a categorical variable into dummy variables:

In [94]:
pd.get_dummies(students.fav_icecream)

,chocolate,strawberry,vanilla
a,0,0,1
b,0,1,0
c,0,1,0
d,0,0,1
e,0,1,0
x,1,0,0


### Group By

_Grouping_ puts together rows according to the values for a certain variable:

In [95]:
# exemplify on a new dataframe
performance = pd.DataFrame([
    ('Alice', 'CS 101', 4.0),
    ('Alice', 'CS 102', 3.0),
    ('Alice', 'EE 201', 4.0),
    ('Bob',   'CS 101', 3.0),
    ('Bob',   'EE 201', 3.9),
], columns=['student', 'class', 'grade'])

In [96]:
performance

,student,class,grade
0,Alice,CS 101,4.0
1,Alice,CS 102,3.0
2,Alice,EE 201,4.0
3,Bob,CS 101,3.0
4,Bob,EE 201,3.9


In order to see the effects of grouping, we apply an aggregation on all the rows for each student:

In [97]:
performance.groupby('student').grade.mean()

student
Alice    3.666667
Bob      3.450000
Name: grade, dtype: float64

---

Group by multiple variables:

In [98]:
students.groupby(['graduated', 'fav_icecream']).age.mean()

graduated  fav_icecream
False      strawberry      22.5
           vanilla         20.0
True       chocolate       22.0
           strawberry      22.0
Name: age, dtype: float64

---

Iterate over the `groupby` object, feature value (student name here) and the rows for that value:

In [99]:
for student, classes in performance.groupby('student'):
    print(student, 'took', len(classes), 'classes, with an average of', classes.grade.mean().round(2))

Alice took 3 classes, with an average of 3.67
Bob took 2 classes, with an average of 3.45


**💪 Exercise**: get the maximum grade of each student in the `performance` dataframe:

In [100]:
performance.groupby('student').grade.max()

student
Alice    4.0
Bob      3.9
Name: grade, dtype: float64

### Pivot

_Pivoting_ "flips" the data according and applies an function. Select a discrete variable for the columns, one for the rows, and you get the unique values for each. Each observations in the original dataframe, falls into one such value intersection. Pick an aggregation to apply to that set of observations.

For example, if want to know the average `height` and `weight` for those students that `graduated` and those that did not:

In [101]:
pd.pivot_table(
    students,
    index='graduated',
    values=['height', 'weight'],
    aggfunc='mean',
)

,height,weight
graduated,,
False,169.75,76.25
True,174.50,82.50


Or the maximum `height` and `weight` instead:

In [102]:
pd.pivot_table(
    students,
    index='graduated',
    values=['height', 'weight'],
    aggfunc='max',
)

,height,weight
graduated,,
False,184,92
True,179,95


---

A special case of pivoting is _cross tabulation_, which returns the counts at each feature value intersection:

In [103]:
pd.crosstab(students.fav_icecream, students.graduated, margins=True)

graduated,False,True,All
fav_icecream,,,
chocolate,0,1,1
strawberry,2,1,3
vanilla,2,0,2
All,4,2,6


**💪 Exercise**: get `mean` of `age` and `height` for students who `can_ride` and those who can't:

In [104]:
pd.pivot_table(
    students,
    index='can_ride',
    values=['age', 'height'],
    aggfunc='mean',
)

,age,height
can_ride,,
False,22.333333,163.333333
True,20.666667,179.333333


### Melt

_Melting_ can be thought of as the reverse of pivoting.

In [105]:
# exemplify on a new dataframe
height_evolution = pd.DataFrame({
    'Alice': np.linspace(160, 190, num=5),
    'Bob':   np.linspace(170, 180, num=5),
    'year':  range(2000, 2005),
})

In [106]:
height_evolution

,Alice,Bob,year
0,160.0,170.0,2000
1,167.5,172.5,2001
2,175.0,175.0,2002
3,182.5,177.5,2003
4,190.0,180.0,2004


Currently, each student has their own column. But that is the same type of information. So, we can melt them into a single `student` column:

In [107]:
melted = height_evolution.melt(
    id_vars='year',
    value_vars=['Alice', 'Bob'],

    var_name='student',
    value_name='height',
)
melted

,year,student,height
0,2000,Alice,160.0
1,2001,Alice,167.5
2,2002,Alice,175.0
3,2003,Alice,182.5
4,2004,Alice,190.0
5,2000,Bob,170.0
6,2001,Bob,172.5
7,2002,Bob,175.0
8,2003,Bob,177.5
9,2004,Bob,180.0


### Join

_Joining_ combines two tables, based on a common variable

In [108]:
height_stats = height_evolution.describe().T[['min', 'max']].rename(columns=lambda c: c + '_height')
height_stats

,min_height,max_height
Alice,160.0,190.0
Bob,170.0,180.0
year,2000.0,2004.0


_Note_: since these are column-wise statistics, we also get the smallest and largest values for `year`.

Both tables have the `student` column, which is the one we join on. For every row where `student` is `"Alice"`, combine the information from the `performance` dataframe (`class` and `grade`) with the information from the the other dataframe (`min_height` and `max_height`).

In [109]:
performance.join(height_stats, on='student')

,student,class,grade,min_height,max_height
0,Alice,CS 101,4.0,160.0,190.0
1,Alice,CS 102,3.0,160.0,190.0
2,Alice,EE 201,4.0,160.0,190.0
3,Bob,CS 101,3.0,170.0,180.0
4,Bob,EE 201,3.9,170.0,180.0


_Note_: since `year` is not among `performance.student` values, the default left join ignores that entry. Learn more about [join types](http://www.sql-join.com/sql-join-types/).

### Time Series

In [110]:
from datetime import datetime

In [111]:
# number of pages read for the first week of classes
pages_read = pd.DataFrame({
    'Alice': np.random.randint(0, 50, size=7),
    'Bob':   np.random.randint(0, 20, size=7),
    'date': pd.date_range('7 Jan 2019', periods=7)
})

pages_read

,Alice,Bob,date
0,34,0,2019-01-07
1,28,11,2019-01-08
2,9,7,2019-01-09
3,43,6,2019-01-10
4,10,10,2019-01-11
5,14,9,2019-01-12
6,36,7,2019-01-13


Comparison operations can be done against a `datetime`-compatible object:

In [112]:
late_start = datetime(year=2019, month=1, day=10)  # it's not fair to start counting that early

In [113]:
pages_read[pages_read.date > late_start]

,Alice,Bob,date
4,10,10,2019-01-11
5,14,9,2019-01-12
6,36,7,2019-01-13


While dates represent specific timepoints (of various granularity), the difference between two such objects is a _time delta_: a duration, not a date:

In [114]:
late_start - pages_read.date.iloc[0]

Timedelta('3 days 00:00:00')

It can be instantiated by parsing natural language:

In [115]:
pd.Timedelta('7 days 5 hours 3 minutes')

Timedelta('7 days 05:03:00')

Timedeltas can be used to offset date objects:

In [116]:
pages_read.date + pd.Timedelta(7, 'd')  # much better, a whole week later

0   2019-01-14
1   2019-01-15
2   2019-01-16
3   2019-01-17
4   2019-01-18
5   2019-01-19
6   2019-01-20
Name: date, dtype: datetime64[ns]

### Hierarchical Indices

In [117]:
enrollment = pd.DataFrame({
    'level':    np.random.choice(['grad', 'undergrad', 'phd'], size=20),
    'school':   np.random.choice(['Viterbi', 'Price', 'Marshall', 'Dornsife'], size=20),
    'students': np.random.randint(200, 5_000, size=20),
    'faculty':  np.random.randint(50,  500,   size=20),
}).drop_duplicates(subset=['level', 'school'])

enrollment

,level,school,students,faculty
0,phd,Price,1020,81
2,grad,Viterbi,4925,461
3,phd,Dornsife,3635,427
4,phd,Viterbi,1945,391
6,phd,Marshall,2990,118
7,undergrad,Marshall,236,344
10,undergrad,Price,2256,89
12,grad,Marshall,977,243
16,undergrad,Dornsife,2030,473
17,grad,Price,2365,199


In [118]:
enrollment.set_index(['school', 'level']).sort_index()

students  faculty
school   level                       
Dornsife phd            3635      427
         undergrad      2030      473
Marshall grad            977      243
         phd            2990      118
         undergrad       236      344
Price    grad           2365      199
         phd            1020       81
         undergrad      2256       89
Viterbi  grad           4925      461
         phd            1945      391

---

In [119]:
locations = pd.DataFrame({
    'Alice': ('San Francisco', 'CA', 'Los Angeles', 'CA'),
    'Bob':   ('Rochester', 'NY', 'Los Angeles', 'CA'),
    'Chris': ('Las Vegas', 'NV', 'Pennsylvania', 'PA'),
}).T

locations.columns = pd.MultiIndex.from_product([
    ['home', 'school'],
    ['city', 'state']
], names=['purpose', 'address'])

locations

purpose           home              school      
address           city state          city state
Alice    San Francisco    CA   Los Angeles    CA
Bob          Rochester    NY   Los Angeles    CA
Chris        Las Vegas    NV  Pennsylvania    PA

Read more about [advanced indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)

### Loading and Saving Dataframes

Data will almost always be loaded from an external source.

Load from JSON (open the file in the file browser to check out the source):

In [120]:
pd.read_json('example_files/objects.json')

,grade,name,year
0,3.90,Alice,2
1,3.80,Bob,3
2,3.85,Chris,1


Load from CSV (again, the file is in the `example_files` folder):

In [121]:
pd.read_csv('example_files/tabular.csv')

,grade,name,year
0,3.90,Alice,2
1,3.80,Bob,3
2,3.85,Chris,1


**ℹ️ Tip**: the CSV format is extremely common. There is a huge number of options available for loading such files. Read more about them [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).

Load directly from an URL, letting Pandas do the downloading (paste that link into your browser to check the source):

In [122]:
pd.read_csv('https://raw.githubusercontent.com/stefan-niculae/viz-workshop/master/example_files/tabular.csv')  # can take a second to load

,grade,name,year
0,3.90,Alice,2
1,3.80,Bob,3
2,3.85,Chris,1


Load directly from an archive, letting Pandas do the uncompressing (find the file and extract it in your local file browser to check the source):

In [123]:
pd.read_csv('example_files/archived.csv.zip')

,grade,name,year
0,3.90,Alice,2
1,3.80,Bob,3
2,3.85,Chris,1


---

Saving data (check the result after running in the file browser):

In [124]:
performance.to_csv('students_performance.csv')

**ℹ️ Tip**: if the index is meaningless (e.g.: just the default sequential one), avoid wasting space and slightly encumbering the reading process by omitting it with `ignore_index=True`.

## Further reading
 - Pandas:
   - [visual cheatsheet](http://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)
   - [cookbook](https://chrisalbon.com/#python)
   - [gotchas](https://pandas.pydata.org/pandas-docs/stable/user_guide/gotchas.html#gotchas)
 
Links to more details about particular concepts are placed at the end of their respective (sub)sections.